In [1]:
! pip install torch timm peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

#  Load the CIFAR-10 Dataset

In [2]:
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# Load the CIFAR-10 dataset from Hugging Face Datasets
dataset = load_dataset("cifar10")

# Define image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

# Apply the transformations and prepare the dataset
def transform_example(example):
    example["img"] = transform(example["img"])
    return example

train_dataset = dataset["train"].map(transform_example)
test_dataset = dataset["test"].map(transform_example)

# Convert to PyTorch DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Adjust batch size as needed
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,  # Adjust batch size as needed
    shuffle=False
)

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

#  Load and Configure the Model with LoRA

In [3]:
import timm
from peft import LoraConfig, get_peft_model
from torch import nn

# Load the pretrained model
model_id = "timm/poolformer_m36.sail_in1k"
num_classes = 10  # CIFAR-10 has 10 classes
model = timm.create_model(model_id, pretrained=True, num_classes=num_classes)

# Identify the layers to apply LoRA to
target_modules_regex = r".*\.mlp\.fc\d"

# Configure LoRA
config = LoraConfig(
    target_modules=target_modules_regex,
    modules_to_save=["head.fc"]  # Ensure the classification head is also updated
)

# Wrap the model with PEFT (LoRA)
peft_model = get_peft_model(model, config)

# Check the number of trainable parameters
peft_model.print_trainable_parameters()

# Move the model to the appropriate device
peft_model.to(device)

model.safetensors:   0%|          | 0.00/225M [00:00<?, ?B/s]

trainable params: 1,067,530 || all params: 56,478,740 || trainable%: 1.8901


PeftModel(
  (base_model): LoraModel(
    (model): MetaFormer(
      (stem): Stem(
        (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
        (norm): Identity()
      )
      (stages): Sequential(
        (0): MetaFormerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): MetaFormerBlock(
              (norm1): GroupNorm1(1, 96, eps=1e-05, affine=True)
              (token_mixer): Pooling(
                (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
              )
              (drop_path1): Identity()
              (layer_scale1): Scale()
              (res_scale1): Identity()
              (norm2): GroupNorm1(1, 96, eps=1e-05, affine=True)
              (mlp): Mlp(
                (fc1): lora.Conv2d(
                  (base_layer): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (

# Define the Training and Evaluation Loops

In [4]:
from torch.optim import Adam

# Define the training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs["img"].to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Define the evaluation loop
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs["img"].to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy

# Train the Model

In [7]:
# Define the training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs, labels = batch["img"].to(device), batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Define the evaluation loop
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch["img"].to(device), batch["label"].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy

In [9]:
# Define the training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        # Unpack the batch
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# Define the evaluation loop
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in dataloader:
            # Unpack the batch
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy

In [10]:
# Save the PEFT model
peft_model.save_pretrained("path/to/save/model")

# Load the model later
from peft import PeftModel
loaded_model = PeftModel.from_pretrained(model, "path/to/save/model", config=config)
loaded_model.to(device)


PeftModel(
  (base_model): LoraModel(
    (model): MetaFormer(
      (stem): Stem(
        (conv): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
        (norm): Identity()
      )
      (stages): Sequential(
        (0): MetaFormerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): MetaFormerBlock(
              (norm1): GroupNorm1(1, 96, eps=1e-05, affine=True)
              (token_mixer): Pooling(
                (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
              )
              (drop_path1): Identity()
              (layer_scale1): Scale()
              (res_scale1): Identity()
              (norm2): GroupNorm1(1, 96, eps=1e-05, affine=True)
              (mlp): Mlp(
                (fc1): lora.Conv2d(
                  (base_layer): Conv2d(96, 384, kernel_size=(1, 1), stride=(1, 1))
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (

In [12]:
# Step 1: Install libraries
!pip install huggingface_hub

# Step 2: Log in to Hugging Face
from huggingface_hub import login

login(token='token')

# Step 3: Save and push your model
from huggingface_hub import HfApi

# Set up the repository
repo_id = "NeuraFusionAI/Fine-tunning-LORA"

# Push the model
peft_model.push_to_hub(repo_id)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


ValueError: Invalid token passed!